In [75]:
import pandas_gbq
import os
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import storage
import numpy as np
import sqlite3
import zipfile



In [79]:

# Paths
file_path = r"C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded"

# Read CSVs in chunks and upload to BigQuery
credentials_path = r"C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project-bt-b14310631abc.json"
credentials = service_account.Credentials.from_service_account_file(credentials_path)
gbq_proj_id = "wedge-project-bt"
dataset_id = "wedge_data"



# # Read CSVs in chunks and upload to BigQuery
# credentials_path = r"C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\umt-msba-gg-key.json"
# credentials = service_account.Credentials.from_service_account_file(credentials_path)
# gbq_proj_id = "umt-msba"
# dataset_id = "wedge_transactions"




In [81]:
chunk_size = 50000  

# Define the function to drop a table if it exists
def drop_table_if_exists(dataset_id, table_name, credentials):
    client = bigquery.Client(credentials=credentials, project=gbq_proj_id)
    table_id = f"{gbq_proj_id}.{dataset_id}.{table_name}"
    try:
        client.delete_table(table_id)  # API request
        print(f"Deleted table '{table_id}'")
    except NotFound:
        print(f"Table '{table_id}' not found, skipping deletion.")

# Define the detect_delimiter function here

def detect_delimiter(filename):
    with open(filename, 'r') as file:
        first_line = file.readline()
        if ";" in first_line:
            return ";"
        else:
            return ","

# Define the clean_dataframe function here

def clean_dataframe(df):
    # Replace "NULL", "\\N", "\\\\N", and blanks with np.NaN
    df.replace(["NULL", "\\N", "\\\\N", ""], np.NaN, inplace=True)
    
    # Trim spaces from string columns
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()
    
    # Convert empty strings to np.NaN
    df = df.replace("", np.NaN)

    # Type Conversion
    float_columns = [
        'register_no', 'emp_no', 'trans_no', 'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice',
        'altPrice', 'tax', 'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype',
        'voided', 'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'numflag',
        'itemstatus', 'tenderstatus', 'varflag', 'local', 'organic', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id'
    ]
    boolean_columns = ['memType', 'staff', 'batchHeaderID', 'display']
    string_columns = ['upc', 'description', 'trans_type', 'trans_subtype', 'trans_status', 'charflag']

    for col in float_columns:
        if col in df.columns:
            df[col] = df[col].astype(float)

    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].astype(bool)

    for col in string_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)

    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

    return df

print(f"Contents of {file_path}:")
print(os.listdir(file_path))

# Loop through all files in the directory
for root, dirs, files in os.walk(file_path):
    for file in files:
        full_path = os.path.join(root, file)
        
        if file.endswith('.csv'):
            print(f"Found CSV file: {file}")
            
            delimiter = detect_delimiter(full_path)
            print(f"Detected delimiter: {delimiter}")
            
            table_name = file.replace('.csv', '')
            
            # Drop the table if it exists
            drop_table_if_exists(dataset_id, table_name, credentials)  # Pass credentials here
            
            print(f"Reading CSV file in chunks: {file}...")
            chunk_iter = pd.read_csv(full_path, delimiter=delimiter, chunksize=chunk_size, dtype=str, low_memory=False)
            
            for idx, chunk_df in enumerate(chunk_iter):
                # Clean the dataframe
                chunk_df = clean_dataframe(chunk_df)
                
                # Modify the field names to comply with BigQuery rules
                chunk_df.columns = [col.lower().replace(';', '') for col in chunk_df.columns]
                
                print(f"Uploading chunk {idx + 1} to {table_name}...")
                if idx == 0:
                    # For the first chunk, create the table
                    pandas_gbq.to_gbq(chunk_df, f"{dataset_id}.{table_name}", project_id=gbq_proj_id, if_exists='replace', credentials=credentials)
                else:
                    # For subsequent chunks, append to the table
                    pandas_gbq.to_gbq(chunk_df, f"{dataset_id}.{table_name}", project_id=gbq_proj_id, if_exists='append', credentials=credentials)
                del chunk_df  # Clear the chunk from memory

print("Upload complete.")




Contents of C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded:
['transArchive_201001_201003.csv', 'transArchive_201004_201006.csv', 'transArchive_201007_201009.csv', 'transArchive_201010_201012.csv', 'transArchive_201101_201103.csv', 'transArchive_201104.csv', 'transArchive_201105.csv', 'transArchive_201106.csv', 'transArchive_201107_201109.csv', 'transArchive_201110_201112.csv', 'transArchive_201201_201203.csv', 'transArchive_201201_201203_inactive.csv', 'transArchive_201204_201206.csv', 'transArchive_201204_201206_inactive.csv', 'transArchive_201207_201209.csv', 'transArchive_201207_201209_inactive.csv', 'transArchive_201210_201212.csv', 'transArchive_201210_201212_inactive.csv', 'transArchive_201301_201303.csv', 'transArchive_201301_201303_inactive.csv', 'transArchive_201304_201306.csv', 'transArchive_201304_201306_inactive.csv', 'transArchive_201307_201309.csv', 'transArchive_201307_201309_inactive.csv', 'transArchive_20

100%|██████████| 1/1 [00:00<00:00, 1005.11it/s]

In [64]:

# # Define the directory where your zip files are located
# zip_dir = "C:\\Users\\britt\\OneDrive - The University of Montana\\Applied Data Analytics\\Wedge Project\\WedgeZipOfZips"

# # Define the directory where you want to save the unzipped files
# extract_to_dir = "C:\\Users\\britt\\OneDrive - The University of Montana\\Applied Data Analytics\\Wedge Project\\wedge-project\\Uploaded"

# # Walk through the directory
# for root, dirs, files in os.walk(zip_dir):
#     for file in files:
#         if file.endswith('.zip'):
#             # Construct the file path
#             file_path = os.path.join(root, file)
#             # Open the zip file
#             with zipfile.ZipFile(file_path, 'r') as zip_ref:
#                 # Extract all the contents into the directory
#                 zip_ref.extractall(extract_to_dir)
#                 print(f"Extracted {file} to {extract_to_dir}")

# print("All files have been extracted.")


Extracted transArchive_201001_201003.zip to C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded
Extracted transArchive_201004_201006.zip to C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded
Extracted transArchive_201007_201009.zip to C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded
Extracted transArchive_201010_201012.zip to C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded
Extracted transArchive_201101_201103.zip to C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded
Extracted transArchive_201104.zip to C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded
Extracted transArchive_201105.zip to C:\Users\britt\OneDrive - The Univ

## Task 1

In [65]:

# Headers for the files
headers = [
    'datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description', 'trans_type', 'trans_subtype', 'trans_status',
    'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 'taxexempt', 'foodstamp',
    'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype', 'voided', 'percentDiscount', 'ItemQtty', 'volDiscType',
    'volume', 'VolSpecial', 'mixMatch', 'matched', 'memType', 'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag',
    'varflag', 'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id'
]

# Loop through all files in the directory
for root, dirs, files in os.walk(file_path):
    for file in files: 
        full_path = os.path.join(root, file) 
        if file.endswith('.csv'): 
            with open(full_path, 'r') as f: 
                first_line = f.readline().strip() 

            # Check if the file likely has headers based on the first line
            if first_line.startswith('datetime'):
                print(f"File {file} seems to already have headers. Skipping...")
                continue

            # If not, then prepend headers to the file
            print(f"Adding headers to {file}")
            with open(full_path, 'r') as f:
                content = f.read()
            with open(full_path, 'w') as f:
                f.write(','.join(headers) + '\n' + content)


In [77]:
chunk_size = 50000  

# Define the function to drop a table if it exists
def drop_table_if_exists(dataset_id, table_name):
    client = bigquery.Client()
    table_id = f"{gbq_proj_id}.{dataset_id}.{table_name}"
    try:
        client.delete_table(table_id)
        print(f"Deleted table '{table_id}'")
    except NotFound:
        print(f"Table '{table_id}' not found, skipping deletion.")

# Define the detect_delimiter function here

def detect_delimiter(filename):
    with open(filename, 'r') as file:
        first_line = file.readline()
        if ";" in first_line:
            return ";"
        else:
            return ","

# Define the clean_dataframe function here

def clean_dataframe(df):
    # Replace "NULL", "\\N", "\\\\N", and blanks with np.NaN
    df.replace(["NULL", "\\N", "\\\\N", ""], np.NaN, inplace=True)
    
    # Trim spaces from string columns
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()
    
    # Convert empty strings to np.NaN
    df = df.replace("", np.NaN)

    # Type Conversion
    float_columns = [
        'register_no', 'emp_no', 'trans_no', 'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice',
        'altPrice', 'tax', 'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype',
        'voided', 'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'numflag',
        'itemstatus', 'tenderstatus', 'varflag', 'local', 'organic', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id'
    ]
    boolean_columns = ['memType', 'staff', 'batchHeaderID', 'display']
    string_columns = ['upc', 'description', 'trans_type', 'trans_subtype', 'trans_status', 'charflag']

    for col in float_columns:
        if col in df.columns:
            df[col] = df[col].astype(float)

    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].astype(bool)

    for col in string_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)

    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

    return df

print(f"Contents of {file_path}:")
print(os.listdir(file_path))

# Loop through all files in the directory
for root, dirs, files in os.walk(file_path):
    for file in files:
        full_path = os.path.join(root, file)
        
        if file.endswith('.csv'):
            print(f"Found CSV file: {file}")
            
            delimiter = detect_delimiter(full_path)
            print(f"Detected delimiter: {delimiter}")
            
            table_name = file.replace('.csv', '')
            
            # Drop the table if it exists
            drop_table_if_exists(dataset_id, table_name)
            
            print(f"Reading CSV file in chunks: {file}...")
            chunk_iter = pd.read_csv(full_path, delimiter=delimiter, chunksize=chunk_size, dtype=str, low_memory=False)
            
            for idx, chunk_df in enumerate(chunk_iter):
                # Clean the dataframe
                chunk_df = clean_dataframe(chunk_df)
                
                # Modify the field names to comply with BigQuery rules
                chunk_df.columns = [col.lower().replace(';', '') for col in chunk_df.columns]
                
                print(f"Uploading chunk {idx + 1} to {table_name}...")
                if idx == 0:
                    # For the first chunk, create the table
                    pandas_gbq.to_gbq(chunk_df, f"{dataset_id}.{table_name}", project_id=gbq_proj_id, if_exists='replace', credentials=credentials)
                else:
                    # For subsequent chunks, append to the table
                    pandas_gbq.to_gbq(chunk_df, f"{dataset_id}.{table_name}", project_id=gbq_proj_id, if_exists='append', credentials=credentials)
                del chunk_df  # Clear the chunk from memory

print("Upload complete.")



Contents of C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded:
['transArchive_201001_201003.csv', 'transArchive_201004_201006.csv', 'transArchive_201007_201009.csv', 'transArchive_201010_201012.csv', 'transArchive_201101_201103.csv', 'transArchive_201104.csv', 'transArchive_201105.csv', 'transArchive_201106.csv', 'transArchive_201107_201109.csv', 'transArchive_201110_201112.csv', 'transArchive_201201_201203.csv', 'transArchive_201201_201203_inactive.csv', 'transArchive_201204_201206.csv', 'transArchive_201204_201206_inactive.csv', 'transArchive_201207_201209.csv', 'transArchive_201207_201209_inactive.csv', 'transArchive_201210_201212.csv', 'transArchive_201210_201212_inactive.csv', 'transArchive_201301_201303.csv', 'transArchive_201301_201303_inactive.csv', 'transArchive_201304_201306.csv', 'transArchive_201304_201306_inactive.csv', 'transArchive_201307_201309.csv', 'transArchive_201307_201309_inactive.csv', 'transArchive_20

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:

chunk_size = 50000  

# Print directory contents for debugging
print(f"Contents of {file_path}:")
print(os.listdir(file_path))

def detect_delimiter(filename):
    with open(filename, 'r') as file:
        first_line = file.readline()
        if ";" in first_line:
            return ";"
        else:
            return ","

def clean_dataframe(df):
    # Replace "NULL", "\\N", "\\\\N", and blanks with np.NaN
    df.replace(["NULL", "\\N", "\\\\N", ""], np.NaN, inplace=True)
    
    # Trim spaces from string columns
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()
    
    # Convert empty strings to np.NaN
    df = df.replace("", np.NaN)

    # Type Conversion
    float_columns = [
        'register_no', 'emp_no', 'trans_no', 'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice',
        'altPrice', 'tax', 'taxexempt', 'foodstamp', 'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype',
        'voided', 'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 'matched', 'numflag',
        'itemstatus', 'tenderstatus', 'varflag', 'local', 'organic', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id'
    ]
    boolean_columns = ['memType', 'staff', 'batchHeaderID', 'display']
    string_columns = ['upc', 'description', 'trans_type', 'trans_subtype', 'trans_status', 'charflag']

    for col in float_columns:
        if col in df.columns:
            df[col] = df[col].astype(float)

    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].astype(bool)

    for col in string_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)

    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

    return df


# Loop through all files in the directory
for root, dirs, files in os.walk(file_path):
    for file in files:
        full_path = os.path.join(root, file)
        
        if file.endswith('.csv'):
            print(f"Found CSV file: {file}")
            
            delimiter = detect_delimiter(full_path)
            print(f"Detected delimiter: {delimiter}")
            
            print(f"Reading CSV file in chunks: {file}...")
            chunk_iter = pd.read_csv(full_path, delimiter=delimiter, chunksize=chunk_size, dtype=str, low_memory=False)
            
            table_name = file.replace('.csv', '')  # Name the table after the CSV file
            table_id = f"{gbq_proj_id}.{dataset_id}.{table_name}"
            
            for idx, chunk_df in enumerate(chunk_iter):
                # Clean the dataframe
                chunk_df = clean_dataframe(chunk_df)
                
                # Modify the field names to comply with BigQuery rules
                chunk_df.columns = [col.lower().replace(';', '') for col in chunk_df.columns]
                
                print(f"Uploading chunk {idx + 1} to {table_name}...")
                if idx == 0:
                    # For the first chunk, create the table
                    pandas_gbq.to_gbq(chunk_df, table_id, project_id=gbq_proj_id, if_exists='replace', credentials=credentials)
                else:
                    # For subsequent chunks, append to the table
                    pandas_gbq.to_gbq(chunk_df, table_id, project_id=gbq_proj_id, if_exists='append', credentials=credentials)
                del chunk_df  # Clear the chunk from memory

print("Upload complete.")


## TASK 2

In [ ]:
# # Path to service account JSON key file
# credentials_path = r"C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\wedge-project-bt-bf0ddf1029cd.json"
# credentials = service_account.Credentials.from_service_account_file(credentials_path)
# client = bigquery.Client(credentials=credentials, project= gbq_proj_id)

# # SQL query
# query = """
# WITH rand_cte AS(
# SELECT DISTINCT card_no
#   FROM `wedge-project-bt.transArchive_*` 
#   WHERE card_no != 3
#   ORDER BY RAND()
#   LIMIT 601)

#   SELECT *
#   FROM `wedge_data.transArchive_*` AS trans
#   JOIN rand_cte 
#   ON rand_cte.card_no = trans.card_no
# """

# # Run the query and get the result as a dataframe
# df = client.query(query).to_dataframe(create_bqstorage_client=False)

# # Save the dataframe to a TXT file
# df.to_csv("output_data1.txt", index=False)


## TASK 3

In [30]:
#Query 1

query = """SELECT
  EXTRACT(DATE
  FROM
    datetime) AS date,
    EXTRACT(HOUR FROM datetime) AS hour,
 ROUND(SUM(total),2) AS spend,
  COUNT(DISTINCT CONCAT(EXTRACT(DATE
        FROM
          datetime), 
          register_no, emp_no, trans_no)) AS trans,
  SUM(CASE
      WHEN trans_status IN ('V', 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `wedge_transactions.transArchive*`
WHERE
  department NOT IN (0,
    15)
  AND (trans_status IS  NULL
  OR trans_status IN (' ','V','R'))
GROUP BY
  date, hour
  ORDER BY 
  date, hour;
  """


In [55]:
# Path to service account JSON key file
conn = sqlite3.connect('wedge-reporting.db')


In [32]:
holder = pandas_gbq.read_gbq(query, project_id=gbq_proj_id, credentials=credentials)    

Downloading: 100%|██████████|


In [33]:
holder.head()

,date,hour,spend,trans,items
0,2010-01-01,9,1006.28,36,245
1,2010-01-01,10,3128.55,82,913
2,2010-01-01,11,4001.66,118,1108
3,2010-01-01,12,3886.51,124,1143
4,2010-01-01,13,4654.52,154,1365


In [34]:
holder.to_sql('date-hour', conn, if_exists='replace', index=False)

39330

In [22]:
#Query 2 

query2 = """SELECT DISTINCT card_no,
   EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime) AS month,
  SUM(total) AS spend,
  COUNT(DISTINCT CONCAT(EXTRACT(DATE
        FROM
          datetime), 
          register_no, emp_no, trans_no)) AS trans,
  SUM(CASE
      WHEN trans_status IN ('V', 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `wedge_transactions.transArchive*`
WHERE
  department NOT IN (0,
    15)
  AND  card_no != 3
  AND (trans_status IS  NULL
  OR trans_status IN (' ','V','R'))
GROUP BY
  card_no, year, month
  ORDER BY 
 card_no, year, month;
  """


In [23]:
holder = pandas_gbq.read_gbq(query2, project_id=gbq_proj_id, credentials=credentials)    

Downloading: 100%|██████████|


In [24]:
holder.head()

,card_no,year,month,spend,trans,items
0,10000.0,2010,10,65.87,4,21
1,10000.0,2010,11,53.12,2,20
2,10000.0,2010,12,17.34,1,6
3,10000.0,2011,1,60.40,4,23
4,10000.0,2011,2,19.65,1,4


In [25]:
holder.to_sql('owner-sales', conn, if_exists='replace', index=False) 

808811

In [56]:
#Query 3 

query3 = """SELECT upc,
  description,
  w.department,
  dept_name,
   EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime) AS month,
  ROUND(SUM(total),2) AS sales,
  COUNT(DISTINCT CONCAT(EXTRACT(DATE
        FROM
          datetime), 
          register_no, emp_no, trans_no)) AS trans,
  SUM(CASE
      WHEN trans_status IN ('V', 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `wedge_transactions.transArchive*` AS w
JOIN `wedge_transactions.department_lookup` AS d
ON w.department = d.department
WHERE
  w.department NOT IN (0,
    15)
  AND  card_no != 3
  AND (trans_status IS  NULL
  OR trans_status IN (' ','V','R'))
GROUP BY
  upc,description,w.department, dept_name, year, month 
ORDER BY 
 sales DESC, description, year, month;
  """


In [57]:
holder = pandas_gbq.read_gbq(query3, project_id=gbq_proj_id, credentials=credentials)    

Downloading: 100%|██████████|


In [58]:
holder.head()

,upc,description,department,dept_name,year,month,sales,trans,items
0,0000000007025,Hot Bar Container,8.0,DELI,2017,1,34137.02,4285,4726
1,0000000007025,Hot Bar Container,8.0,DELI,2016,12,33595.31,4215,4643
2,0003338320038,O.Strawberries 16oz pkg.,2.0,PRODUCE,2010,5,32702.24,6630,7592
3,0000000007025,Hot Bar Container,8.0,DELI,2016,11,31601.92,3982,4397
4,0000000000338,O.Blueberries WI pkg.,2.0,PRODUCE,2011,8,31457.30,5213,5536


In [59]:
holder.to_sql('product-sales', conn, if_exists='replace', index=False) 

1130901

In [60]:
# Close the connection at end of all queries

conn.close()